# Alternative GrabData and SQL Database
Getting exchange data is simple enough using APIs and pre-existing Python wrappers. However, sometimes there is data that is not as easy to find/record. Some sites offer limited/slow/un-stable APIs while others may not even have APIs at all. In order to collect this kind of data we need to develop alternative collection codes. Now that a toddler can pip install TensorFlow the data is arguably a lot more important than the model. Many people can create a neural network and having the skills to make a good aren't hard to learn, but having the data sets to do so has become increasingly more of an important factor.

## Methods
For these reasons we will use basic Get Requests and Selenium to fetch data from websites that do not offer more user friendly tools/interfaces. Then the data will be recorded to an AWS SQL Database.

### Dependencies

In [1]:
import json
import os
import time
import datetime
import numpy as np
import pandas as pd
import requests
import traceback
import smtplib
from email.mime.text import MIMEText
from selenium import webdriver
from selenium.webdriver.support import ui
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
engine = create_engine('sqlite://', echo=False)

### Get Requests
The first website to collect data from offered low level APIs by way of HTTP Get Requests which would return a JSON with the corresponding data. Below is the code for a simple request which is fixed. The second part shows how the URL could be altered to fetch different data according to a list of predefined items we want to collect (this method isn't guaranteed to work, because tinkering with the URL is not always supported, but I've found more often than not it functions well).

In [2]:
URL = 'https://www.website.com'
item = requests.get(url=URL).json()
item = item['data']['value']

items = [thing1, thing2, thing3, thing4, thing5]
for i in items:
    specific_item = items[i]
    URL = 'https://www.website.com/{}'.format(specific_item)
    item = requests.get(url=URL).json()
    item = item['data']['value']

### Selenium Data Retrieval
The second method is to use Selenium to automate web browsing and grab information directly from a web page. For this method we first need a web driver to do the browsing. I prefer FireFox, but the whole list of 3rd party drivers can be found [here](https://docs.seleniumhq.org/download/). When running on a server the code is run headless, which just means without a graphic display as we are used to. However, when first writing the code it is easiest to test it running with a display to visually find desired elements and verify that it is running as expected.  

Using Selenuim there are 3 main funcionalities that are used: clicking, typing, and retrieving. Below is code to navigate to the initial webpage and then code for the 3 functionalities are shown. By combining these together in different ways we can emulate human behavior of going to a website logging in, making queries, and finding information.

As for finding the specific webpage elements to click, type into, and copy there are a couple options, but I find xpath to be my favorite. In FireFox F12 opens Developer Tools and using the inspector we can easily click on the item we want to interact with and copy/paste the associated xpath into the script.  

It should be noted that many websites have tools to block this (eg CAPTCHA). Since it is too difficult to write a code to solve CAPTCHA, the next best best is not write a script that triggers a CAPTCHA in the first place. In my experience refreshing and navigating to different webpages at high speeds with no mouse movement is a surefire way to get a CAPTCHA. On the otherhand Java based interfaces like dropdown menus and typing in text to get an output are relatively safe as they don't need to make further server requests. It doesn't hurt though to put in a couple time.sleep(x) of some variable length which can be quite short and arbitrary mouse movements.

In [3]:
options = Options()
options.headless = True # toggle headless option here

driver = webdriver.Firefox(options=options)

driver.get('https://www.website.com/')

# find an element and click it
element = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[3]/ul/li[6]/a').click()

# find an input box and type into it
element = driver.find_element_by_xpath('//*[@id="user_email"]')
element.send_keys('you@selenium.com')

# find element/output and retrieve data
element = driver.find_element_by_xpath('/html/body/div[1]/main/div/div[2]/div[1]/input').get_attribute('value')

# to avoid CAPTCHAS
# time.sleep(0.5)
# actions =  ActionChains(driver)
# element = driver.find_element_by_xpath('/html/body/div[1]/div[5]/a/strong')
# actions.move_to_element(element).perform()
# OR
# actions.move_by_offset(x, y)

### SQL Database Connection
Now that we have our data we would like to store it on our SQL database, which in this case is hosted on AWS. First we establish a connection.

In [4]:
engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, db_name), poolclass=NullPool)
connection = engine.connect()

### Create a Table
Using pandas we can easily record a DataFrame to the database as a table using df.to_sql().

In [5]:
now = datetime.datetime.now()
headings = ["timestamp", "feature1", "feature2", "feature3"]
data = [now, element1, element2, element3]
df = pd.DataFrame(np.zeros((1, 4)), columns=headings)
df.loc[0, :] = data
df.to_sql("Data_from_{}.format(website_name), con=connection)

### Add Rows
If we are collecting a large set of data we can simply use the above code with a DataFrame containing all the information we need. In other instances (eg time series data) we can add a row to our table as data comes in or at regular intervals.

In [6]:
sql_statement = 'INSERT INTO \"{}\" (timestamp, feature1, feature2, feature3) VALUES ({}, {}, {}, {});'.format(tableName, data[0], data[1], data[2], data[3])
connection.execute(sql_statement)